In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import os
import numpy as np

# Dataset 220x20

mypath = r'C:\Users\marco\ANN_Calibration_FSCAV\CNN'

X = np.load(os.path.join(mypath, 'CNNdataset.npz'))['a']
y = np.load(os.path.join(mypath, 'CNNlabels.npz'))['a']

In [17]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#Dataset 220x220

pathcp = r'C:\Users\marco\ANN_Calibration_FSCAV\CNN\ResNet50\Colorplots'
pathlab = r'C:\Users\marco\ANN_Calibration_FSCAV\CNN\ResNet50\Labels'


X_filenames = [file for file in os.listdir(pathcp)]
Y_filenames = [file for file in os.listdir(pathlab)]

CP = [] #colorplots
LABELS = [] #labels

# labels are in a folder with other files, I select olny labels
for file in Y_filenames:
    if file.find('labels') != -1 :
        labels = np.load(os.path.join(pathlab, file))['a']
        for label in labels :
                LABELS.append(int(float(label)))
for file in X_filenames:
    if file.find('np') != -1 :
        cps = np.load(os.path.join(pathcp, file))['a']
        for cp in cps:
            #for layer in range(3):
            #    standard_scaler = preprocessing.StandardScaler()
            #    cp[:,:,layer] = standard_scaler.fit_transform(cp[:,:,layer])
            CP.append(cp)

# Check length
if len(CP) != len(LABELS):
    print('Error, different lengths')
    
CP = np.array(CP)
LABELS = np.array(LABELS)

In [2]:
CP_train, CP_test, Labels_train, Labels_test = train_test_split(
    CP, LABELS, shuffle = True, test_size=0.3, random_state=1)

In [45]:
import time
from tensorflow.keras.callbacks import TensorBoard

NAME = 'CP-VGG10-220x20-600-100{}'.format(int(time.time()))

tensorboard = TensorBoard(log_dir = r'C:\Users\marco\ANN_Calibration_FSCAV\CNN\VGG10_cust\logs/{}'.format(NAME))

In [46]:
CP = CP[:,:,:20,0]
CP = np.reshape(CP,(-1,220,20,1))

In [47]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization

# Conv + Pooling
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = (220,20,1),padding = 'SAME' )) # (3,3) is window size
model.add(Conv2D(64, (3,3), padding = 'SAME' )) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128, (3,3), padding = 'SAME' ))
model.add(Conv2D(128, (3,3), padding = 'SAME' ))        
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
          
model.add(Conv2D(256, (3,3), padding = 'SAME' ))
model.add(Conv2D(256, (3,3), padding = 'SAME' ))
#model.add(Conv2D(128, (3,3)))         
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(600))
model.add(Activation('relu'))
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('linear'))

model.compile(loss = 'mean_absolute_error', optimizer = tf.keras.optimizers.Adam(0.001), metrics = [tf.keras.metrics.RootMeanSquaredError()])
model.fit(CP,LABELS,batch_size = 64,epochs = 50,  validation_split = 0.25, callbacks = [tensorboard])

Epoch 1/50
90/90 [==============================] - 278s 3s/step - loss: 31.7160 - root_mean_squared_error: 40.8137 - val_loss: 26.1272 - val_root_mean_squared_error: 33.0352
Epoch 2/50
90/90 [==============================] - 253s 3s/step - loss: 22.8524 - root_mean_squared_error: 30.1648 - val_loss: 21.7071 - val_root_mean_squared_error: 29.1840
Epoch 3/50
90/90 [==============================] - 232s 3s/step - loss: 19.1374 - root_mean_squared_error: 25.3856 - val_loss: 25.7884 - val_root_mean_squared_error: 34.1084
Epoch 4/50
90/90 [==============================] - 213s 2s/step - loss: 16.4258 - root_mean_squared_error: 22.6065 - val_loss: 17.6783 - val_root_mean_squared_error: 24.4735
Epoch 5/50
90/90 [==============================] - 207s 2s/step - loss: 14.1931 - root_mean_squared_error: 19.8695 - val_loss: 14.4752 - val_root_mean_squared_error: 21.9521
Epoch 6/50
90/90 [==============================] - 207s 2s/step - loss: 13.1200 - root_mean_squared_error: 18.3707 - val_los

Epoch 48/50
90/90 [==============================] - 260s 3s/step - loss: 3.5926 - root_mean_squared_error: 6.9149 - val_loss: 13.0977 - val_root_mean_squared_error: 20.1521
Epoch 49/50
90/90 [==============================] - 260s 3s/step - loss: 3.6370 - root_mean_squared_error: 6.8499 - val_loss: 12.0809 - val_root_mean_squared_error: 18.4045
Epoch 50/50
90/90 [==============================] - 261s 3s/step - loss: 3.7987 - root_mean_squared_error: 6.8580 - val_loss: 13.1768 - val_root_mean_squared_error: 20.3666


In [48]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 220, 20, 64)       640       
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 220, 20, 64)       36928     
_________________________________________________________________
activation_37 (Activation)   (None, 220, 20, 64)       0         
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 110, 10, 64)       0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 110, 10, 128)      73856     
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 110, 10, 128)      147584    
_________________________________________________________________
activation_38 (Activation)   (None, 110, 10, 128)    